In [1]:
import pygame
import math
import os
import threading
import time
import random

# --- Pygame Initialization ---
pygame.init()

# Screen dimensions
SCREEN_WIDTH = 1280
SCREEN_HEIGHT = 800
SCREEN = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption("Gesture-Controlled Drifting Car")

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
GREEN = (0, 255, 0)
RED = (255, 0, 0)

pygame 2.6.1 (SDL 2.28.4, Python 3.12.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# Load assets (make sure these files are in the same directory as your script)
try:
    CAR_IMAGE_ORIGINAL = pygame.image.load(os.path.join('assets', 'car.png')).convert_alpha()
    road_images_paths = [
        'road_1.png', 
        'road_2.png',
        'road_3.png',
    ]
    ROAD_IMAGES = [
        pygame.transform.scale(
            pygame.image.load(os.path.join('assets', path)).convert_alpha(),
            (SCREEN_WIDTH, SCREEN_HEIGHT)
        ) for path in road_images_paths
    ]

except pygame.error as e:
    print(f"Error loading assets: {e}")
    print("Please make sure 'car.png' and 'road.png' are in an 'assets' folder.")
    pygame.quit()
    exit()

# Scale the car image (adjust as needed)
CAR_IMAGE_ORIGINAL = pygame.transform.scale(CAR_IMAGE_ORIGINAL, (50, 100)) # width, height
# ROAD_IMAGE = pygame.transform.scale(ROAD_IMAGE, (SCREEN_WIDTH, SCREEN_HEIGHT)) # Scale road to screen width

# Game clock
CLOCK = pygame.time.Clock()
FPS = 60

In [3]:
# --- Car Class ---
class Car(pygame.sprite.Sprite):
    def __init__(self, image, x, y):
        super().__init__()
        self.original_image = image
        self.image = self.original_image # Car image is not rotated in this simplified game
        self.rect = self.image.get_rect(center=(x, y))
        self.pos = pygame.math.Vector2(x, y)
        self.speed = 0 # Forward/backward speed
        
        # Physics constants (tune these for feel)
        self.acceleration_rate = 0.5
        self.deceleration_rate = 0.5
        self.max_speed = 100
        self.reverse_speed = 2 # Slower reverse speed
        self.horizontal_speed = 6 # How fast the car moves sideways
        self.dodge_speed_multiplier = 1.8 # How much faster it dodges with 'OK' gestures
        
        # Collision mask for more accurate collision detection
        self.mask = pygame.mask.from_surface(self.image) 

    def update(self, current_gesture, game_state):
        if game_state != GameState.PLAYING:
            self.speed = 0
            return

        # 1. Handle forward/reverse speed
        # Apply friction first
        self.speed *= 0.98 

        if current_gesture == 'accelerate':
            self.speed += self.acceleration_rate
        elif current_gesture == 'reverse':
            self.speed -= self.acceleration_rate * 0.5 # Slower reverse
        elif current_gesture == 'handbrake_calibrate':
            self.speed *= 0.8 # Braking reduces speed
            
        # Clamp speed
        self.speed = max(-self.reverse_speed, min(self.speed, self.max_speed)) 
        if abs(self.speed) < 0.1: # Stop if speed is negligible
            self.speed = 0

        # 2. Handle horizontal (left/right) movement and dodging
        applied_horizontal_speed = self.horizontal_speed

        if current_gesture == 'left_ok_sign': # Left dodge
            applied_horizontal_speed *= self.dodge_speed_multiplier
            self.pos.x -= applied_horizontal_speed
        elif current_gesture == 'right_ok_sign': # Right dodge
            applied_horizontal_speed *= self.dodge_speed_multiplier
            self.pos.x += applied_horizontal_speed
        elif current_gesture == 'left_turn': # Regular left turn
            self.pos.x -= applied_horizontal_speed
        elif current_gesture == 'right_turn': # Regular right turn
            self.pos.x += applied_horizontal_speed
            
        # 3. Keep car within horizontal screen bounds (road edges)
        # These values define the visible road area. Adjust them based on your 'road.jpg'.
        road_left_edge = SCREEN_WIDTH * 0.35 
        road_right_edge = SCREEN_WIDTH * 0.65 

        # Account for the car's width so the *edge* of the car hits the boundary
        half_car_width = self.rect.width / 2

        if self.pos.x + half_car_width > road_right_edge:
            self.pos.x = road_right_edge - half_car_width
        if self.pos.x - half_car_width < road_left_edge:
            self.pos.x = road_left_edge + half_car_width

        # In this model, the car's y-position is fixed on the screen.
        self.rect.center = (int(self.pos.x), int(self.pos.y))
        
        # The car image is not rotated in this simplified game, so self.image remains original_image.


class TrafficCar(pygame.sprite.Sprite):
    def __init__(self, image_list, x, y, base_speed_increase=0): # Added base_speed_increase
        super().__init__()
        self.image = random.choice(image_list)
        self.rect = self.image.get_rect(center=(x, y))
        # Traffic car speed now includes the difficulty-based increase
        self.speed = random.uniform(2, 4) + base_speed_increase # Give each car a random speed + difficulty
        self.mask = pygame.mask.from_surface(self.image) # Collision mask

    def update(self, road_scroll_speed):
        # Traffic moves down the screen. Its speed is relative to the player's.
        self.rect.y += road_scroll_speed + self.speed

        # Remove the car if it goes off the bottom of the screen to save memory
        if self.rect.top > SCREEN_HEIGHT:
            self.kill() # Remove from sprite group

In [4]:
# --- Game State Management ---
class GameState:
    CALIBRATING = 0
    READY = 1
    PLAYING = 2
    GAME_OVER = 3

current_game_state = GameState.CALIBRATING
last_gesture = None # Store the last detected gesture for persistence
gesture_confidence_threshold = 0.7 # Minimum confidence for a gesture to be registered (adjust as needed)

is_game_running = False

# --- Shared Variable for YOLO Output (simulated for now) ---
# In your actual integration, this will be updated by the YOLO thread.
# For now, we'll use keyboard inputs to simulate gesture detections.
shared_gesture_data = {'gesture': None, 'confidence': 0.0}

def simulate_yolo_thread():
    """
    This function simulates the YOLO thread. In your final project,
    this will be replaced by your actual YOLO inference code.
    It continuously updates shared_gesture_data based on keyboard presses.
    """
    global shared_gesture_data

    while not is_game_running:
        time.sleep(0.1)

    while True:
        # Simulate detection with keyboard inputs
        keys = pygame.key.get_pressed()
        
        commands = [] # a list to handle multiple commands at once
        
        # Handle forward/backward movement
        if keys[pygame.K_w]:
            commands.append('accelerate')
        elif keys[pygame.K_s]:
            commands.append('handbrake_calibrate')
        elif keys[pygame.K_r]:
            commands.append('reverse')

        # Handle turning independently
        if keys[pygame.K_a]:
            commands.append('left_turn')
        elif keys[pygame.K_d]:
            commands.append('right_turn')

        # Handle drifting independently
        if keys[pygame.K_q]:
            commands.append('left_ok_sign')
        elif keys[pygame.K_e]:
            commands.append('right_ok_sign')

        # Update the shared data (this is simplified, a real system might be more complex)
        # For now, we can prioritize actions: drift > turn > move
        final_gesture = None
        if 'left_ok_sign' in commands: final_gesture = 'left_ok_sign'
        elif 'right_ok_sign' in commands: final_gesture = 'right_ok_sign'
        elif 'left_turn' in commands: final_gesture = 'left_turn'
        elif 'right_turn' in commands: final_gesture = 'right_turn'
        elif 'accelerate' in commands: final_gesture = 'accelerate'
        elif 'reverse' in commands: final_gesture = 'reverse'
        elif 'handbrake_calibrate' in commands: final_gesture = 'handbrake_calibrate'
            
        shared_gesture_data = {'gesture': final_gesture, 'confidence': 0.95 if final_gesture else 0.0}

        time.sleep(0.05)

# Start the simulated YOLO thread
yolo_thread = threading.Thread(target=simulate_yolo_thread, daemon=True)
yolo_thread.start()

In [5]:
# --- High Score and Asset Loading ---

HIGHSCORE_FILE = "highscore.txt"

def save_highscore(score):
    """Saves the high score to a file."""
    try:
        with open(HIGHSCORE_FILE, "w") as f:
            f.write(str(int(score)))
    except IOError:
        print(f"Error: Unable to save high score to {HIGHSCORE_FILE}")

def load_highscore():
    """Loads the high score from a file."""
    try:
        with open(HIGHSCORE_FILE, "r") as f:
            return int(f.read())
    except (IOError, ValueError):
        # Return 0 if the file doesn't exist or contains invalid data
        return 0

# This list was missing, which would cause the next crash.
# It's better to define it here, outside the game loop.
try:
    traffic_car_images = [
        pygame.transform.scale(pygame.image.load(os.path.join('assets', 'traffic_car_1.png')).convert_alpha(), (50, 100)),
        pygame.transform.scale(pygame.image.load(os.path.join('assets', 'traffic_car_2.png')).convert_alpha(), (50, 100)),
        pygame.transform.scale(pygame.image.load(os.path.join('assets', 'traffic_car_3.png')).convert_alpha(), (50, 100)),
        pygame.transform.scale(pygame.image.load(os.path.join('assets', 'traffic_car_4.png')).convert_alpha(), (50, 100))
        # You can add more traffic car images here, e.g., 'traffic_car_2.png'
    ]
except pygame.error as e:
    print(f"Error loading traffic car assets: {e}")
    pygame.quit()
    exit()

In [6]:
# --- Text Display Utility ---
def draw_text(surface, text, size, color, x, y, center=True):
    font = pygame.font.Font(None, size)
    text_surface = font.render(text, True, color)
    text_rect = text_surface.get_rect()
    if center:
        text_rect.center = (x, y)
    else:
        text_rect.topleft = (x, y)
    surface.blit(text_surface, text_rect)

# --- Game Loop ---
def game_loop():
    global current_game_state, last_gesture, shared_gesture_data

    is_game_running = True

    car_start_y = SCREEN_HEIGHT - 150
    car = Car(CAR_IMAGE_ORIGINAL, SCREEN_WIDTH // 2, car_start_y)
    traffic_group = pygame.sprite.Group()

    # Game variables
    road_y_offset = 0
    current_score = 0
    high_score = load_highscore()
    
    # Calibration variables
    calibration_start_time = 0
    calibration_duration = 3
    calibration_gesture_detected = False

    # Difficulty scaling variables
    difficulty_level = 0
    score_for_next_level = 500
    base_traffic_speed_increase = 0
    base_min_spawn_time_ms = 700
    base_max_spawn_time_ms = 2000
    
    # Define lanes for traffic
    LANE_CENTERS = [
        SCREEN_WIDTH * 0.38,
        SCREEN_WIDTH * 0.44,
        SCREEN_WIDTH * 0.50, # Center lane
        SCREEN_WIDTH * 0.56,
        SCREEN_WIDTH * 0.62
    ]

    SPAWN_TRAFFIC_EVENT = pygame.USEREVENT + 1
    
    running = True
    while running:
        # --- Single, reliable event loop ---
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            
            if event.type == SPAWN_TRAFFIC_EVENT and current_game_state == GameState.PLAYING:
                lane = random.choice(LANE_CENTERS)
                new_traffic_car = TrafficCar(traffic_car_images, lane, -100, base_traffic_speed_increase)
                traffic_group.add(new_traffic_car)

                min_spawn_time = max(200, base_min_spawn_time_ms - difficulty_level * 100)
                max_spawn_time = max(600, base_max_spawn_time_ms - difficulty_level * 200)
                random_delay = random.randint(min_spawn_time, max_spawn_time)
                pygame.time.set_timer(SPAWN_TRAFFIC_EVENT, random_delay, loops=1)
            
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_ESCAPE:
                    running = False
                
                # Handle Game Over options
                if current_game_state == GameState.GAME_OVER:
                    if event.key == pygame.K_SPACE:
                        # Restart the game by calling the main loop again
                        game_loop()
                    if event.key == pygame.K_0:
                        # Set running to False to exit the game
                        running = False

                if event.key == pygame.K_SPACE and current_game_state == GameState.READY:
                    # Start the game if SPACE is pressed
                    current_game_state = GameState.PLAYING
                    current_score, difficulty_level, base_traffic_speed_increase = 0, 0, 0
                    car = Car(CAR_IMAGE_ORIGINAL, SCREEN_WIDTH // 2, car_start_y)
                    traffic_group.empty()
                    pygame.time.set_timer(SPAWN_TRAFFIC_EVENT, 100, loops=1) # Start traffic

                if event.key == pygame.K_c:
                    # Reset to calibration screen
                    current_game_state = GameState.CALIBRATING
                    calibration_gesture_detected, calibration_start_time = False, 0
                    traffic_group.empty()
                    car = Car(CAR_IMAGE_ORIGINAL, SCREEN_WIDTH // 2, car_start_y)
                    current_score, difficulty_level, base_traffic_speed_increase = 0, 0, 0

        # --- Gesture Processing ---
        detected_gesture = shared_gesture_data['gesture']
        current_gesture_command = detected_gesture if shared_gesture_data['confidence'] >= gesture_confidence_threshold else None

        # --- Game State Logic ---
        if current_game_state == GameState.CALIBRATING:
            SCREEN.fill(BLACK)
            
            # Draw the static helper text
            draw_text(SCREEN, "CALIBRATING: Show Both Fists Closed", 40, WHITE, SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2 - 50)
            draw_text(SCREEN, "Hold the gesture to begin.", 30, WHITE, SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2)
            draw_text(SCREEN, "(Simulated by holding the 'S' key)", 25, WHITE, SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2 + 40)
            
            # --- Corrected Timer Logic ---
            # Check if the calibration gesture is currently active
            if current_gesture_command == 'handbrake_calibrate':
                # If this is the first moment we detect the gesture, start the timer
                if not calibration_gesture_detected:
                    calibration_start_time = time.time()
                    calibration_gesture_detected = True
                
                # Now that we know the timer has started, we can calculate the elapsed time
                elapsed_time = time.time() - calibration_start_time
                
                # Check if the timer has finished
                if elapsed_time >= calibration_duration:
                    current_game_state = GameState.READY
                    # Reset the flag so calibration can be run again later
                    calibration_gesture_detected = False 
                else:
                    # If the timer is still running, draw the green countdown text
                    countdown = int(calibration_duration - elapsed_time) + 1
                    draw_text(SCREEN, f"Calibrating in {countdown}...", 50, GREEN, SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2 + 100)
            else:
                # If the gesture is NOT active (e.g., key is released), reset the detection flag.
                # This is important so the timer can start fresh next time.
                calibration_gesture_detected = False

        elif current_game_state == GameState.READY:
            # (Your ready screen drawing code is fine)
            SCREEN.fill(BLACK)
            draw_text(SCREEN, "READY TO PLAY!", 50, GREEN, SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2 - 50)
            draw_text(SCREEN, "Show 'accelerate' gesture or press SPACE to start", 30, WHITE, SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2 + 20)

            # Start game with gesture
            if current_gesture_command == 'accelerate':
                current_game_state = GameState.PLAYING
                current_score, difficulty_level, base_traffic_speed_increase = 0, 0, 0
                car = Car(CAR_IMAGE_ORIGINAL, SCREEN_WIDTH // 2, car_start_y)
                traffic_group.empty()
                pygame.time.set_timer(SPAWN_TRAFFIC_EVENT, 100, loops=1) # Start traffic

        elif current_game_state == GameState.PLAYING:
            # --- Difficulty and Scoring Logic (Your logic was good, just integrated here) ---
            if current_score >= score_for_next_level:
                difficulty_level += 1
                score_for_next_level += 500 + (difficulty_level * 200)
                base_traffic_speed_increase += 0.5
            
            road_scroll_speed = max(0, car.speed) / 2
            current_score += car.speed / 10 if car.speed > 0 else 0.1 # Score for surviving
            
            # Updates
            car.update(current_gesture_command, current_game_state)
            traffic_group.update(road_scroll_speed)

            # Collision Check
            if pygame.sprite.spritecollide(car, traffic_group, False, pygame.sprite.collide_mask):
                current_game_state = GameState.GAME_OVER
                if current_score > high_score:
                    high_score = current_score
                    save_highscore(high_score)

            # Drawing
            level_index = (difficulty_level // 2) % len(ROAD_IMAGES)
            current_road_image = ROAD_IMAGES[level_index]
            
            road_y_offset = (road_y_offset + road_scroll_speed) % SCREEN_HEIGHT
            SCREEN.fill(BLACK)
            SCREEN.blit(current_road_image, (0, road_y_offset - SCREEN_HEIGHT))
            SCREEN.blit(current_road_image, (0, road_y_offset))
            traffic_group.draw(SCREEN)
            SCREEN.blit(car.image, car.rect)
            draw_text(SCREEN, f"Score: {int(current_score)}", 30, (200, 200, 255), SCREEN_WIDTH - 120, 20, center=True)
            draw_text(SCREEN, f"High Score: {int(high_score)}", 25, (200, 200, 255), SCREEN_WIDTH - 120, 60, center=True)
            draw_text(SCREEN, f"Level: {difficulty_level}", 25, WHITE, SCREEN_WIDTH - 120, 100, center=True)

            speed_kph = int(car.speed * 12) # Multiply by a factor to make it look like KPH
            draw_text(SCREEN, f"Speed: {speed_kph} KPH", 25, WHITE, SCREEN_WIDTH - 120, 140, center=True)

        elif current_game_state == GameState.GAME_OVER:
            traffic_group.empty()
            SCREEN.fill(BLACK)
            draw_text(SCREEN, "GAME OVER!", 60, RED, SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2 - 100)
            draw_text(SCREEN, f"Your Score: {int(current_score)}", 40, WHITE, SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2 - 20)

            draw_text(SCREEN, "Press 'C' to Restart", 30, GREEN, SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2 + 60)
            draw_text(SCREEN, "Press '0' to Exit", 30, WHITE, SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2 + 100)
            
        pygame.display.flip()
        CLOCK.tick(FPS)
    is_game_running = False
    pygame.quit()
    exit()

In [ ]:
if __name__ == '__main__':
    # Create an 'assets' directory if it doesn't exist
    if not os.path.exists('assets'):
        os.makedirs('assets')
        print("Created 'assets' directory. Please place image files inside it.")
        exit()

    # Set the game running flag to True BEFORE calling the game loop.
    # This allows the simulation thread to start detecting keys.
    is_game_running = True
    
    # Start the game
    game_loop()

    # This part is technically not reached if the game exits internally,
    # but it's good practice for other potential scenarios.
    is_game_running = False

Exception in thread Thread-5 (simulate_yolo_thread):
Traceback (most recent call last):
  File "c:\Users\VICTUS\anaconda3\Lib\threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "c:\Users\VICTUS\anaconda3\Lib\threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\VICTUS\AppData\Local\Temp\ipykernel_26620\2542709759.py", line 32, in simulate_yolo_thread
pygame.error: video system not initialized


error: display Surface quit

: 